# Dependences

In [30]:
using CUDA
using DelimitedFiles
using Test
using BenchmarkTools

# initial data
R_Agg = 13
r_max = 3.0
n_knn = 50
idx_red_size =  r_max ≤ 2.80 ? 13 :
            2.80 < r_max ≤ 3.45 ? 21 :
            3.45 < r_max ≤ 3.80 ? 39 :
            3.80 < r_max ≤ 4.00 ? 55 :
            70
X = Float64.(readdlm("../../../data/init/Sphere/$(R_Agg).xyz")[3:end,2:end]) |> cu

# kNN
idx      = Int32.(zeros(size(X, 1), size(X, 1))) |> cu;
idx_sum  = Int32.(zeros(1, size(idx, 1))) |> cu
idx_red  = Int32.(zeros(idx_red_size, size(idx, 1))) |> cu
idx_contractile =Int32.(zeros(n_knn,size(X,1))) |> cu

# forces
force = CUDA.zeros(size(X));

# Generic Functions

In [31]:
# force_func
cubic(A,r_max,r_min,dist) = - A * (dist-r_max)^2* (dist-r_min)

# distances
euclidean(points,i,j) = sqrt((points[i,1]-points[j,1])^2+(points[i,2]-points[j,2])^2+(points[i,3]-points[j,3])^2)

euclidean (generic function with 1 method)

# idx

## functions

In [32]:
function dist_kernel!(idx, points ,r_max)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    
    # Limiting data inside matrix
    if i <= size(points, 1) && j <= size(points, 1)
        if euclidean(points,i,j) < r_max
            idx[i, j] = i
        else
            idx[i, j] = 0
        end 
    end
    return nothing
end

function reduce_kernel(idx,idx_red,idx_sum)
    # Defining Index for kernel
    i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

    # Limiting data inside matrix
    if i <= size(idx,1)
        # Cleaning idx_sum
        idx_sum[i] = 0
        
        # looping on each row for searching non-zero values
        for j = 1:size(idx,1)
            if idx[j,i] != 0
                idx_sum[i] += 1
                idx_red[idx_sum[i],i] = j
            end
        end
    end
    
    return nothing
end

function index_contractile!(idx_contractile,idx_sum,idx_red)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(idx_contractile, 1) && j <= size(idx_contractile,2)
        idx_contractile[i,j] = idx_red[rand(1:idx_sum[j]),j]
    end
    return nothing
end

function nearest_neighbors(idx, idx_red, idx_sum, idx_contractile, points ,r_max)
    # Calculating Distance Matrix
    threads =(32,32)
    blocks  =cld.(size(points,1),threads)
    @cuda threads=threads blocks=blocks dist_kernel!(idx, points ,r_max)

    # Reducing Distance Matrix to Nearest Neighbors
    threads=1024
    blocks=cld.(size(idx,1),threads)
    @cuda threads=threads blocks=blocks reduce_kernel(idx,idx_red,idx_sum)

    # Finding index contractile
    threads =(32,32)
    blocks  =cld.(size(X,1),threads)
    @cuda threads=threads blocks=blocks index_contractile!(idx_contractile,idx_sum,idx_red)
end

nearest_neighbors (generic function with 1 method)

## running

In [33]:
println("---------------------------------- VARIABLES ------------------------------------")
println("R_Agg = $(R_Agg) | R_Max = $(r_max) | col_size_idx = $(idx_red_size)")
display(X)
println("------------------------------------ SIZES --------------------------------------")
println("Size → idx     = $(size(idx))")
println("     → idx_sum = $(size(idx_sum))")
println("     → idx_red = $(size(idx_red))")
println("------------------------------------ RESULTS ------------------------------------")
@btime nearest_neighbors(idx, idx_red, idx_sum,idx_contractile, X ,r_max)
println("----------------------------------- IDX RED -------------------------------------")
display(idx_red)
println("----------------------------------- IDX SUM -------------------------------------")
display(idx_sum)
println("------------------------------- IDX CONTRACTILE ---------------------------------")
display(idx_contractile)
println("---------------------------------- Memory Used ----------------------------------")
CUDA.memory_status()  

---------------------------------- VARIABLES ------------------------------------
R_Agg = 13 | R_Max = 3.0 | col_size_idx = 21


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.5  -4.04  -12.25
  1.5  -4.04  -12.25
 -3.5  -2.31  -12.25
 -1.5  -2.31  -12.25
  0.5  -2.31  -12.25
  2.5  -2.31  -12.25
 -2.5  -0.58  -12.25
 -0.5  -0.58  -12.25
  1.5  -0.58  -12.25
  3.5  -0.58  -12.25
 -3.5   1.15  -12.25
 -1.5   1.15  -12.25
  0.5   1.15  -12.25
  ⋮           
  0.5  -1.15   12.25
  2.5  -1.15   12.25
 -2.5   0.58   12.25
 -0.5   0.58   12.25
  1.5   0.58   12.25
  3.5   0.58   12.25
 -3.5   2.31   12.25
 -1.5   2.31   12.25
  0.5   2.31   12.25
  2.5   2.31   12.25
 -0.5   4.04   12.25
  1.5   4.04   12.25

------------------------------------ SIZES --------------------------------------
Size → idx     = (1620, 1620)
     → idx_sum = (1, 1620)
     → idx_red = (21, 1620)
------------------------------------ RESULTS ------------------------------------
  13.349 μs (77 allocations: 4.03 KiB)
----------------------------------- IDX RED -------------------------------------


21×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  1   1   3   1   1   2   3   4   5  …  1580  1581  1582  1583  1589  1590
  2   2   4   3   2   5   4   5   6     1581  1582  1583  1584  1590  1591
  4   5   7   4   4   6   7   7   8     1582  1583  1584  1585  1591  1592
  5   6  23   5   5   9   8   8   9     1588  1589  1590  1591  1596  1597
 19  20  29   7   6  10  11   9  10     1589  1590  1591  1592  1597  1598
 24  25  30   8   8  26  12  12  13  …  1595  1596  1597  1598  1601  1602
 25  26  35  24   9  32  30  13  14     1611  1611  1612  1613  1616  1617
 30  31  36  30  25  33  36  31  32     1615  1612  1613  1614  1617  1618
 31  32  37  31  31  38  37  37  38     1616  1615  1616  1617  1619  1619
 32  33   0  36  32  39  43  38  39        0  1616  1617  1618  1620  1620
  0   0   0  37  37  40  44  44  45  …     0  1617  1618  1620     0     0
  0   0   0  38  38   0  45  45  46        0  1619  1619     0     0     0
  0   0   0   0  39   0   0  46  47        0     0

----------------------------------- IDX SUM -------------------------------------


1×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 10  10  9  12  13  11  12  13  13  10  …  13  13  10  9  12  13  11  10  10

------------------------------- IDX CONTRACTILE ---------------------------------


50×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  2   6   3   1  31  40  36   9  32  …  1582  1619  1616  1583  1619  1598
 19  20  35   1  38  40   8  46   5     1581  1582  1590  1585  1590  1619
  2  32  30   7  38  38   3  38  38     1611  1612  1582  1583  1596  1598
 24  20   7  24  38   5  44   9  10     1615  1582  1613  1613  1620  1602
 30   2  30   5  39   5   8   5   5     1595  1611  1616  1598  1591  1617
 19   6  36   4  31  33   8   5  14  …  1611  1619  1597  1614  1617  1620
  1  25   4  38   6  10  12  44  32     1611  1583  1617  1592  1616  1620
  2  33   3   3   6  39  11  31  13     1581  1596  1613  1613  1620  1597
 19   6  29  30  38  10  11  44  38     1611  1582  1591  1618  1601  1598
 31   1   3  37   8  40   4  13  32     1580  1596  1582  1598  1590  1597
  4  33   4  38   6  38  43  13  46  …  1595  1619  1590  1614  1597  1618
  1  26  36  31   8  32   7  38  32     1589  1590  1617  1583  1620  1592
  2   1   7  24   2  26   3  12   6     1595  1619

---------------------------------- Memory Used ----------------------------------
Effective GPU memory usage: 22.89% (458.375 MiB/1.955 GiB)
No memory pool is in use.

# forces

## functions

In [34]:
function sum_force!(idx,points,force, force_func,A,r_max,r_min)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(points, 1) && k <= size(points, 2)

        # Cleaning idx_sum
        force[i,k] = 0
        dist = 0

        # Iterate on each row
        for j=1:size(idx,1)

            # Finding forces
            if idx[j,i] != i && idx[j,i] != 0
                dist = euclidean(points,i,idx[j,i])
                force[i,k] +=force_func(A,r_max,r_min,dist) * points[idx[j,i],k] / dist
            end
            
        end

        # Adding Contrractile Force
        if idx_contractile[t_knn,i] != i && idx_contractile[t_knn,i] != 0
            dist = euclidean(points,i,idx_contractile[t_knn,i])
            force[i,k] = A_con*points[idx_contractile[t_knn,i],k]/dist
        end
        
    end
    return nothing
end

sum_force! (generic function with 3 methods)

## running

In [35]:
dt = 0.5
A     = 1
A_con = 0.1
r_min = 2.0
r_max = r_max

t_knn = 1

1

In [36]:
# GPU
threads =(100,3)
blocks  =cld.(size(X,1),threads)
@btime @cuda threads=threads blocks=blocks sum_force!(idx_red,idx_contractile, X ,force, cubic,A,r_max,r_min,A_con,Int(t_knn))
force

  7.063 μs (48 allocations: 2.58 KiB)


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.075      -0.202      -0.6125
  0.125111   -0.115603   -0.613044
  0.120219    0.0792914   0.360268
 -0.0250222  -0.20218    -0.613044
 -0.0249191  -0.144033   -0.528784
  0.15875    -0.0405695  -0.374298
 -0.174684   -0.0573961  -0.529541
  0.075      -0.029      -0.6125
  0.052948   -0.102013   -0.374519
  0.224593   -0.0573961  -0.529541
 -0.075       0.0575     -0.6125
 -0.124571    0.144004   -0.610398
 -0.0249548   0.0289476  -0.529541
  ⋮                      
  0.0747426  -0.144004    0.610398
  0.0748644  -0.0289476   0.529541
  0.0715554  -0.0139363  -0.297831
 -0.0881945  -0.0204611   0.374298
  0.0250222  -0.0575511   0.613044
  0.194028   -0.0204611   0.374298
 -0.0529167   0.0405695   0.374298
 -0.0250222   0.20218     0.613044
 -0.075       0.1155      0.6125
  0.0176389   0.0405695   0.374298
  0.0155762  -0.130495   -0.327601
  0.124596    0.230253    0.528784

## Pending (Test with CPU)

In [39]:
# CPU
X_CPU = Matrix(X)
idx_red_CPU = Matrix(idx_red)
idx_contractile_CPU = Matrix(idx_contractile)
force_CPU = zeros(size((X)))
# @btime for j = 1:size(idx_red_CPU,1)
#     for i =1:size(X,1)
#         if idx_red_CPU[j,i] != 0 && idx_red_CPU[j,i] != i
#             dist = euclidean(X_CPU,i,idx_red_CPU[j,i])
#             force_CPU[i,:] += cubic.(A,r_max,r_min,dist) .* X_CPU[idx_red_CPU[j,i],:] ./ dist
#         end
#     end
# end

# testing
# @test Matrix(force) ≈ force_CPU atol=0.01

1620×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0